# LKA urbanization review
http://wiki.worldbank.org/display/GEOS/LKA_SSAU1_ColomboUrbanization

Will perform the following analyses:
1. Extract urban areas
2. Summarize nighttime lights
3. Summarize built area change
4. Run Market access  
   a. Airports  
   b. Ports  
   c. Schools  
   d. Hospitals  

In [38]:
import sys, os, importlib
import rasterio, boto3
import reverse_geocode

import geopandas as gpd
import pandas as pd
import skimage.graph as graph

# Import GOST libraries; sys.path.append will be unnecessary if libraries are already installed
sys.path.append("../../../../gostrocks/src")
sys.path.append("../../../../GOST_Urban")
sys.path.append("../../../../GOSTNets_Raster/src")

import GOSTNets_Raster.market_access as ma
import GOSTRocks.rasterMisc as rMisc
import src.UrbanRaster as urban
from GOSTRocks.misc import tPrint
%matplotlib inline  

In [11]:
iso3 = 'LKA'

out_folder = "/home/wb411133/temp/%s" % iso3
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

global_friction = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"

# Define local files 
friction_file = os.path.join(out_folder, "friction_surface.tif")
focal_admin2 = os.path.join(out_folder, "admin.shp")
wp_1km = os.path.join(out_folder, "WP_2020_1km.tif")
urban_extents = os.path.join(out_folder, "urban_extents.shp")
hd_urban_extents = os.path.join(out_folder, "hd_urban_extents.shp")
airports = os.path.join(out_folder, "airports.shp")
ports = os.path.join(out_folder, "ports.shp")

if not os.path.exists(friction_file):
    rMisc.clipRaster(rasterio.open(global_friction), gpd.read_file(focal_admin2), friction_file)

# Calculate urban extents from 1km WorldPop
if not os.path.exists(urban_extents):
    urban_shp = wp.calculateUrban(rasterio.open(wp_1km), smooth=False)
    urban_shp.to_file(urban_extents)

# Calculate urban extents from 1km WorldPop
if not os.path.exists(hd_urban_extents):
    urban_r = urban.urbanGriddedPop(wp_1km)
    urban_ext = urban_r.calculateUrban(densVal=300, totalPopThresh=5000, smooth=False, queen=False)
    urban_ext.to_file(urban_extents)
    hd_urban_ext = urban_r.calculateUrban(densVal=1500, totalPopThresh=50000, smooth=True, queen=True)
    hd_urban_ext.to_file(hd_urban_extents)

../../../../GOST_Urban/src/UrbanRaster.py:290: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  xxGeom['geometry '] = xxGeom.buffer((popRaster.res[0] / 2))


In [19]:
#Summarize nighttime lights
bucket = "wbgdecinternal-ntl"
s3 = boto3.client('s3')
viirs_annual = s3.list_objects(Bucket=bucket, Prefix = 'NTL/VIIRS/Annual/VIIRS_ANNUAL_EOG/')
viirs_files = [os.path.join(f's3://{bucket}/', x['Key']) for x in viirs_annual['Contents']]

urb_ext = gpd.read_file(urban_extents)
hd_urb_ext = gpd.read_file(hd_urban_extents)
inAdm = gpd.read_file(focal_admin2)

for viirs_file in viirs_files:
    yr = viirs_file.split("_")[-4]
    curR = rasterio.open(viirs_file)
    # Summarize VIIRS in urban extents
    res = rMisc.zonalStats(urb_ext, curR, minVal=0.1, allTouched=True)
    res = pd.DataFrame(res, columns=['SUM','MIN','MAX','MEAN'])
    urb_ext[f'v_{yr}'] = res['SUM']
    # Summarize VIIRS in hd urban extents
    res = rMisc.zonalStats(hd_urb_ext, curR, minVal=0.1, allTouched=True)
    res = pd.DataFrame(res, columns=['SUM','MIN','MAX','MEAN'])
    hd_urb_ext[f'v_{yr}'] = res['SUM']
    # Summarize VIIRS in admin areas
    res = rMisc.zonalStats(inAdm, curR, minVal=0.1, allTouched=False)
    res = pd.DataFrame(res, columns=['SUM','MIN','MAX','MEAN'])
    inAdm[f'v_{yr}'] = res['SUM']

In [25]:
pd.DataFrame(urb_ext).drop(['geometry'], axis=1).to_csv(os.path.join(out_folder, "urban_viirs.csv"))
pd.DataFrame(hd_urb_ext).drop(['geometry'], axis=1).to_csv(os.path.join(out_folder, "hd_urban_viirs.csv"))
pd.DataFrame(inAdm).drop(['geometry'], axis=1).to_csv(os.path.join(out_folder, "adm_viirs.csv"))

In [31]:
# summarize built area change
global_ghsl = "/home/public/Data/GLOBAL/GHSL/ghsl.vrt"
inR = rasterio.open(global_ghsl)
if urb_ext.crs != inR.crs:
    urb_ext = urb_ext.to_crs(inR.crs)
if hd_urb_ext.crs != inR.crs:
    hd_urb_ext = hd_urb_ext.to_crs(inR.crs)
if inAdm.crs != inR.crs:
    inAdm = inAdm.to_crs(inR.crs)

ex_cols = [0,1,2,3,4,5,6]
col_names=['cNA','H20','notBuilt','b2014','b2000','b1990','b1975']

# Summarize GHSL in urban extents
urb_res = rMisc.zonalStats(urb_ext, inR, rastType='C', unqVals=ex_cols)
urb_res = pd.DataFrame(urb_res, columns=col_names)
urb_res['ID'] = urb_ext['ID']

# Summarize GHSL in hd urban extents
hd_urb_res = rMisc.zonalStats(hd_urb_ext, inR, rastType='C', unqVals=ex_cols)
hd_urb_res = pd.DataFrame(hd_urb_res, columns=col_names)
hd_urb_res['ID'] = hd_urb_ext['ID']

# Summarize GHSL in admin areas
adm_res = rMisc.zonalStats(inAdm, inR, rastType='C', unqVals=ex_cols)
adm_res = pd.DataFrame(adm_res, columns=col_names)
adm_res['ID'] = inAdm['WB_ADM2_CO']


In [34]:
urb_res.to_csv(os.path.join(out_folder, "urban_ghsl.csv"))
hd_urb_res.to_csv(os.path.join(out_folder, "hd_urban_ghsl.csv"))
adm_res.to_csv(os.path.join(out_folder, "adm_ghsl.csv"))

In [37]:
# Run market access
inR = rasterio.open(friction_file)
frictionD = inR.read()[0,:,:] * 1000
mcp = graph.MCP_Geometric(frictionD)

port_data = gpd.read_file(ports)
airport_data = gpd.read_file(airports)
urb_centroids = urb_ext.copy()
urb_centroids['geometry'] = urb_centroids['geometry'].apply(lambda x: x.centroid)
hd_urb_centroids = hd_urb_ext.copy()
hd_urb_centroids['geometry'] = hd_urb_centroids['geometry'].apply(lambda x: x.centroid)

In [90]:
importlib.reload(ma)

popR = rasterio.open(wp_1km)
inR = rasterio.open(friction_file)
frictionD = inR.read()[0,:,:] * 1000
mcp = graph.MCP_Geometric(frictionD)
    
port_access = ma.summarize_travel_time_populations(popR, inR, port_data, mcp, inAdm, os.path.join(out_folder, 'port_tt.tif'))
airport_access = ma.summarize_travel_time_populations(popR, inR, airport_data, mcp, inAdm, os.path.join(out_folder, 'airport_tt.tif'))
urban_access = ma.summarize_travel_time_populations(popR, inR, urb_centroids, mcp, inAdm, os.path.join(out_folder, 'urban_tt.tif'))
hd_urban_access = ma.summarize_travel_time_populations(popR, inR, hd_urb_centroids, mcp, inAdm, os.path.join(out_folder, 'hd_urban_tt.tif'))

In [97]:
pd.DataFrame(port_access.drop(['geometry'], axis=1)).to_csv(os.path.join(out_folder, "port_access.csv"))
pd.DataFrame(airport_access.drop(['geometry'], axis=1)).to_csv(os.path.join(out_folder, "airport_access.csv"))
pd.DataFrame(urban_access.drop(['geometry'], axis=1)).to_csv(os.path.join(out_folder, "urban_access.csv"))
pd.DataFrame(hd_urban_access.drop(['geometry'], axis=1)).to_csv(os.path.join(out_folder, "hd_urban_access.csv"))